# downloading the dataset

In [3]:
# # !pip install kaggle
# !mkdir ~/.kaggle
# # download kaggle.json (from kaggle->account->settings->api->new_Token) and upload in colab first
# !cp /content/kaggle.json ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json
# # # https://www.kaggle.com/datasets/himanshusahu00738/face-emotions-expressions
# !kaggle datasets download -d himanshusahu00738/face-emotions-expressions
# # relpace the name with downladed zip file name
# !unzip face-emotions-expressions.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
face-emotions-expressions.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  face-emotions-expressions.zip
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# importing Libraries

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [5]:
tf.__version__

'2.12.0'

# Data Preprocessing

## Preprocessing the Training set

In [6]:
target_size=(48, 48)
batch_size=32

trainValidation_data_generator = ImageDataGenerator(
  # rescale=1./225,
  rotation_range=10,  # Randomly rotate images by up to 10 degrees
  zoom_range=0.2,  # Randomly zoom images by up to 20%
  width_shift_range=0.1,  # Randomly shift images horizontally by up to 10% of the image width
  height_shift_range=0.1,  # Randomly shift images vertically by up to 10% of the image height
  shear_range=0.2,  # Randomly apply shearing transformations
  horizontal_flip=True,  # Randomly flip images horizontally
  fill_mode='nearest',  # Fill in missing pixels with the nearest value
  # preprocessing_function=grayscale_RGB_and_upsizing,
  validation_split=0.2    # set the validation split                                
)
training_set = trainValidation_data_generator.flow_from_directory(
                                                            directory='/content/training', 
                                                            target_size=target_size,
                                                            batch_size=batch_size,
                                                            class_mode='categorical', 
                                                            color_mode='rgb', 
                                                            shuffle=True, 
                                                            subset='training' # set as training data
                                                        )

Found 45936 images belonging to 7 classes.


## Preprocessing the Test set

In [7]:
test_data_generator = ImageDataGenerator(
    # rescale = 1./255
)
test_set = test_data_generator.flow_from_directory(
  directory='/content/test', 
  target_size=target_size,
  class_mode='categorical', 
  color_mode='rgb',
  shuffle=False, 
  batch_size=batch_size
)

Found 14356 images belonging to 7 classes.


# Data Analysis

In [8]:
class_dictionary = training_set.class_indices
class_keys   = list(training_set.class_indices.keys())
class_values = list(training_set.class_indices.values())
class_count  = len(class_keys)

In [9]:
print(class_dictionary  )
print(class_keys  )
print(class_values)
print(class_count )

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
[0, 1, 2, 3, 4, 5, 6]
7


# Buliding CNN

In [10]:
from tensorflow import keras
from sklearn import set_config
from sklearn.metrics import confusion_matrix, classification_report
from keras.preprocessing.image import ImageDataGenerator as ImgDataGen
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.models import Sequential, load_model

from keras import layers, optimizers, metrics, regularizers, models
from keras.optimizers import Adam, Adamax
# from tensorflow.keras.metrics import categorical_crossentropy, sparse_categorical_crossentropy

In [11]:
# define the image shape for the input layer
input_shape=(target_size[0], target_size[1], 3)
batch_size = batch_size
epochs=80
# ask_epoch=0

In [12]:
from keras.models import Model
model_name='EfficientNetB3'
base_model=tf.keras.applications.efficientnet.EfficientNetB3(
  include_top=False, 
  weights="imagenet",
  input_shape=input_shape, 
  pooling='max'
)

# Let's make our base_model trainable to get better results
base_model.trainable=True
x=base_model.output

x=BatchNormalization(
  axis=-1, 
  momentum=0.99, 
  epsilon=0.001,
  name='batch_norm_x' 
)(x)

x = Dense(
  256, 
  kernel_regularizer = regularizers.l2(l = 0.016),
  activity_regularizer=regularizers.l1(0.006),
  bias_regularizer=regularizers.l1(0.006),
  activation='relu',
  name='dense_x'
)(x)

x=Dropout(
  rate=.4, 
  seed=123,
  name='dropout_x'
)(x)  
      
output=Dense(
  class_count, 
  activation='softmax',
  name='dense_output'
)(x)

cnn_model=Model(inputs=base_model.input, outputs=output, name=model_name)
learning_rate=.001 # start with this learning rate
cnn_model.compile(
  Adamax(learning_rate=learning_rate), 
  loss='categorical_crossentropy', 
  metrics=['accuracy']
)

43941136/43941136 [==============================] - 2s 0us/step


# Training the CNN

In [13]:
def checkpoint_path(model_name):
    checkpoint_path = "/content/models/"+model_name+".ckpt"
    return checkpoint_path

check_point = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_path(model_name),
  save_weights_only=True,
  save_best_only=True, 
  monitor="val_accuracy",
  verbose = 1,
)

reduce_learning = tf.keras.callbacks.ReduceLROnPlateau(
  monitor="val_accuracy", 
  # factor=0.5, 
  patience=2,
  verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
  monitor="val_accuracy", 
  min_delta=0.0005,
  patience=11, 
  verbose=1,
  restore_best_weights=True
)
callbacks = [
            check_point,
            reduce_learning, 
            # early_stop, 
          ]

In [14]:
import os
# starting the Checkpoint for the model
e_checkpoint_path = "/content/models/fedav_best_model.h5"
checkpoint_dir = os.path.dirname(e_checkpoint_path)

e_check_point = tf.keras.callbacks.ModelCheckpoint(
                                          filepath=e_checkpoint_path,
                                          save_weights_only=True,
                                          save_best_only=True, 
                                          monitor="val_accuracy",
                                          verbose = 1,
                                        )

e_reduce_learning = tf.keras.callbacks.ReduceLROnPlateau(
                                            monitor="val_accuracy", 
                                            # factor=0.5, 
                                            patience=2,
                                            verbose=1
                                            )

e_early_stop = tf.keras.callbacks.EarlyStopping(
                                       monitor="val_accuracy", 
                                       min_delta=0.0005,
                                       patience=11, 
                                       verbose=1,
                                       restore_best_weights=True
                                       )

e_callbacks = [
#             e_check_point,
            e_reduce_learning, 
            # e_early_stop, 
          ]

In [15]:
cnn_model.fit(
  x=training_set,  
  validation_data= test_set,
  steps_per_epoch=training_set.n//training_set.batch_size,
  epochs=epochs, 
  verbose=1, 
  callbacks=callbacks,  
  #   validation_steps=None, 
  validation_steps= test_set.n//test_set.batch_size,
) 

Epoch 1/80
1435/1435 [==============================] - ETA: 0s - loss: 4.5404 - accuracy: 0.3650
Epoch 1: val_accuracy improved from -inf to 0.47726, saving model to /content/models/EfficientNetB3.ckpt
1435/1435 [==============================] - 176s 106ms/step - loss: 4.5404 - accuracy: 0.3650 - val_loss: 2.0607 - val_accuracy: 0.4773 - lr: 0.0010
Epoch 2/80
1435/1435 [==============================] - ETA: 0s - loss: 1.6745 - accuracy: 0.4927
Epoch 2: val_accuracy improved from 0.47726 to 0.54143, saving model to /content/models/EfficientNetB3.ckpt
1435/1435 [==============================] - 139s 97ms/step - loss: 1.6745 - accuracy: 0.4927 - val_loss: 1.3948 - val_accuracy: 0.5414 - lr: 0.0010
Epoch 3/80
1435/1435 [==============================] - ETA: 0s - loss: 1.4034 - accuracy: 0.5423
Epoch 3: val_accuracy improved from 0.54143 to 0.56166, saving model to /content/models/EfficientNetB3.ckpt
1435/1435 [==============================] - 140s 97ms/step - loss: 1.4034 - accuracy:

### saving the trained model and loading it

In [ ]:
# # use below line to save the trained model 
# cnn_model.save('/content/drive/MyDrive/DataBasesML/face_emotions/model.h5')

# ## use below line to use the trained model in seperate file
# # loaded_model = tf.keras.models.load_model('path/to/save/model.h5')

## evaluate model

In [23]:
model = cnn_model 
loss, acc = model.evaluate(training_set, verbose=2)
loss, acc = model.evaluate(test_set, verbose=2)

# Loads the weights
cnn_model.load_weights(checkpoint_path(model_name=model_name))

# Re-evaluate the model
loss, acc = cnn_model.evaluate(test_set, verbose=2)



1436/1436 - 70s - loss: 0.5919 - accuracy: 0.8286 - 70s/epoch - 49ms/step
449/449 - 10s - loss: 1.0790 - accuracy: 0.6631 - 10s/epoch - 22ms/step
449/449 - 10s - loss: 1.0561 - accuracy: 0.6659 - 10s/epoch - 22ms/step


## Part 4 - Making a single prediction

In [45]:
import numpy as np
import keras.utils as image

test_image = image.load_img('/content/single/9.jpg', target_size=(48, 48))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn_model.predict(test_image)
class_indices = training_set.class_indices
print(class_indices)

predicted_class_index = np.argmax(result[0])
for key, val in class_indices.items():
        if val == predicted_class_index:
            predicted_class = key

print(predicted_class," ",predicted_class_index)

1/1 [==============================] - 0s 90ms/step
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
surprise   6
